****Импортируем все необходимые библиотеки.****

In [1]:
#Сразу подключим мониторинг
!pip install openai llama-index-core "arize-phoenix[evals,llama-index]" gcsfs nest-asyncio "openinference-instrumentation-llama-index>=2.0.0"

INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of google-api-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine 

In [2]:
#Загрузим наш фреймворк для взаимодействия самую свежую версию
!pip install openai llama_index

In [5]:
#Установим улучшенный поисковик для модели
!pip install llama-index-retrievers-bm25

  Using cached llama_index_core-0.12.14-py3-none-any.whl.metadata (2.5 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.7/646.7 kB 40.0 MB/s eta 0:00:00
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.0.post1
    Uninstalling llama-index-core-0.11.0.post1:
      Successfully uninstalled llama-index-core-0.11.0.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.11.0 requires llama-index-core==0.11.0.post1, but you have llama-index-core 0.12.14 which is incompatible.
llama-index-agent-openai 0.3.3 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-cor

****Подготовка данных и формирование базы данных для RAG****

In [6]:
#Сохраним наш токен для модели и для Lora-Saiga
import getpass # для работы с паролями
import os      # для работы с окружением и файловой системой

# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")
#os.environ['HuggingFace_API_KEY'] = getpass.getpass("Введите токен от HuggingFace:")

Введите OpenAI API Key: ········


In [9]:
!pip install kaggle
!kaggle datasets download -d PromptCloudHQ/imdb-data



Dataset URL: https://www.kaggle.com/datasets/PromptCloudHQ/imdb-data
License(s): other
  0%|                                                | 0.00/134k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 134k/134k [00:00<00:00, 70.4MB/s]


In [10]:
!unzip -qo "imdb-data.zip"

In [11]:
import pandas as pd

df = pd.read_csv('IMDB-Movie-Data.csv')

In [12]:
df = df.drop(['Rank', 'Year', 'Revenue (Millions)', 'Metascore', 'Runtime (Minutes)'], axis=1)


In [15]:
import re
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\b(a|the|an)\b', '', text)  # remove articles
        text = re.sub(r'\s+', ' ', text).strip() #remove multiple spaces
        return text
    else:
        return text

df = df.applymap(preprocess_text)

<ipython-input-15-c75a4bf1cdc4>:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(preprocess_text)


In [16]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.6 MB/s eta 0:00:00:00:0100:01


In [17]:
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True,
    bnb_4bit_quant_type="nf4"
)


In [19]:
films = df.values.tolist()

In [20]:
from llama_index.core import Document
documents = []
for movie_data in films:
    title, genres, description, director, actors, rating, votes = movie_data
    doc = Document(
        text=f"Title: {title}\nDescription: {description}",
        metadata={
            "title": title,
            "genres": genres,
            "director": director,
            "actors": actors,
            "rating": rating,
            "votes": votes
        }
    )
    documents.append(doc)

#index = VectorStoreIndex.from_documents(documents)

**# **Интерфейс модели RAG и ее инференс.****

In [21]:
!pip install llama-index-embeddings-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.14
    Uninstalling llama-index-core-0.12.14:
      Successfully uninstalled llama-index-core-0.12.14
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.11.0 requires llama-index-core==0.11.0.post1, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-retrievers-bm25 0.5.2 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.


In [23]:
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core import Document, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding()

class MyRetriever:
    def __init__(self, document, top_k=10):
       self.document = document
       self.top_k = top_k

    def retrieve(self, query):
        index = VectorStoreIndex.from_documents(documents)
        retriever = QueryFusionRetriever(
            [
                index.as_retriever(similarity_top_k=self.top_k),
                BM25Retriever.from_defaults(docstore=index.docstore, similarity_top_k=self.top_k)
            ],
            num_queries=1,
            use_async=False
        )
        nodes = retriever.retrieve(query)
        return [node.node.text for node in nodes]


In [94]:
import torch
torch.cuda.empty_cache()
   


In [95]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
device = torch.device("cuda")
MODEL_NAME = 'IlyaGusev/saiga_mistral_7b'
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = DEFAULT_SYSTEM_PROMPT = """Ты - Фил, дружелюбный русскоязычный ассистент по подбору фильмов. 

Твоя задача - помогать пользователям выбирать фильмы, основываясь на их запросах. 

**Начало разговора:**

•   Представься пользователю в начале диалога, сообщив, что ты Фил и готов помочь с выбором фильма.
•   Спроси, какие у пользователя предпочтения или какой фильм они ищут.

**Анализ запроса:**

•   Внимательно анализируй запрос пользователя.
•   Если в запросе упоминается актер, очень внимательно проверь фамилию, так как существует много похожих фамилий.
•   Определи ключевые критерии запроса: жанр, актеры, год выпуска, режиссер, тема и т.д.
•   **Используй предоставленную тебе базу знаний о фильмах (далее – "каталог фильмов") для ответа на вопросы.**.
•   Не придумывай информацию.

**Ответ:**

•   Если в "каталоге фильмов" есть информация, подходящая под запрос пользователя, предоставь ее.
•   Если нет подходящей информации в "каталоге фильмов", то честно скажи: "Извините, у меня нет информации по вашему запросу".
•   Отвечай четко и по делу, избегая лишней информации.

**Пример:**

•   **Пользователь:** "Привет, хочу фильм с Томом Хэнксом"
•   **Фил:** "Привет! Я Фил, ваш помощник по подбору фильмов. Я с удовольствием помогу вам найти фильм с Томом Хэнксом. Что-нибудь еще вас интересует, например, жанр или год выпуска?"

**Важно:**
•   Не придумывай ничего и всегда опирайся на предоставленную базу знаний о фильмах.
•   Будь вежливым и приветливым.
"""
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE,
        retriever = None
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]
        self.retriever = retriever # Добавляем retriever


    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })


    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

    def add_retriever_message(self, user_message, retrieve_limit = 1):
      if self.retriever:
        query_engine = RetrieverQueryEngine(self.retriever)
        context = self.retriever.retrieve(user_message)
        context_message = "Используй следующую информацию для ответа: " + " ".join(context)
        self.messages.append({
                "role": "user",
                "content": context_message
            })

def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

# Загрузка модели и токенизатора (ваш код)
offload_folder = "/content/offload" # Указываем путь для выгрузки
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    offload_folder=offload_folder,
    torch_dtype=torch.float16,
    device_map='auto'

)

model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map='auto'
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)
inputs = ["Посоветуй несколько фильмов жанра Fantasy про волшебника","Посоветуй все фильмы о которых знаешь, где играет Matthew Macfadyen","Посоветуй несколько фильмов с Morgan Freeman"]
retriever = MyRetriever(documents) # Создаем экземпляр ретривера
for inp in inputs:
    conversation = Conversation(retriever = retriever)
    conversation.add_user_message(inp)
    conversation.add_retriever_message(inp, retrieve_limit = 1) # Добавляем контекст ретривера в сообщение
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}

[NodeWithScore(node=TextNode(id_='a57865b4-d3af-4ce8-ac9c-3588492266ba', embedding=None, metadata={'title': "pan's labyrinth", 'genres': 'drama,fantasy,war', 'director': 'guillermo del toro', 'actors': 'ivana baquero, ariadna gil, sergi lópez,maribel verdú', 'rating': 8.2, 'votes': 498879}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='81f51f42-cb50-42a2-915e-adc02f424c67', node_type='4', metadata={'title': "pan's labyrinth", 'genres': 'drama,fantasy,war', 'director': 'guillermo del toro', 'actors': 'ivana baquero, ariadna gil, sergi lópez,maribel verdú', 'rating': 8.2, 'votes': 498879}, hash='b3356173a52739220d0d84bdae7d924a1f545a43fdedd63bd54b12aa902be566')

****В целом модель дает довольно сносные ответы,но также время от времени присутствуют галлюцинации. К сожалению в Kaggle не работает трассировка llama-phoenix и причины галлюцинирования изучить не удается. В качестве модели была использована Saiga + LoRa адаптер. В качестве поисковика BM25 ретривер,который очень полезен для такой задачи включая поиск по словам. В качестве постобработки решил не использовать Реранкеры или lingua модель,т.к особо прироста в производительности это не дало, а GPU нагружает не плохо. В качестве датасета был использован датасет с фильмами для рекомендации пользователю фильмов на вечер по его запросу. Модель нуждается в безусловной доработке. Лучше всего работают запросы если совмещать русский и английский языки в частности сложности перевода фамилий и прочего.В итоге получилась простенька RAG система на борту с русскоязычной моделью Сайга.****